In [ ]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from ipywidgets import FloatProgress
import wandb

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/scnet/scnet_s101_fpn_carafe.py')

PREFIX = '../../input/data/'

# wandb.init()
# wandb.run.name = 'faster_rcnn_x101_32x4d_fpn_1x_coco'
# wandb.run.save()
# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = [(512, 512), (1024,1024)]
cfg.data.train.pipeline[2]['multiscale_mode'] = 'range'


cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/scnet_s101_fpn_carafe_ms'
# cfg.model.roi_head.bbox_head.num_classes = 11
# cfg.model.bbox_head.num_classes = 11
# print(cfg.model.roi_head.bbox_head)
# cfg.model.roi_head.mask_head.num_classes=11
cfg.resume_from = './work_dirs/scnet_s101_fpn_carafe_ms/epoch_17.pth'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [ ]:
model = build_detector(cfg.model)

In [ ]:
datasets = [build_dataset(cfg.data.train)]

In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)